In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from time import sleep
import fake_useragent

ua = fake_useragent.UserAgent()
text = 'Machine Learning'

def get_url():
    for page in range(1,3):
        print(f'Log: working over page {page}...')
        url = f'https://career.habr.com/vacancies?page={page}&q={text}&type=all'
        response = requests.get(url, headers = {'user-agent': ua.random})
        soup = BeautifulSoup(response.content, 'lxml')
        data = soup.find_all('a', attrs = {'class': 'vacancy-card__icon-link'})
        for i in data:
            resume_url = 'https://career.habr.com/' + i.get('href')
            yield resume_url

def get_resume():
    for resume in get_url():
        response = requests.get(resume, headers = {'user-agent': ua.random})
        soup = BeautifulSoup(response.content, 'lxml')
        data = soup.find('article', class_= 'vacancy-show')
        
        try:
            title = data.find(class_='page-title__title').text
        except:
            title = ''
        try:
            employer = data.find(class_ = 'basic-salary basic-salary--appearance-vacancy-header').text
        except:
            employer = ''
        try:
            req = data.find_all(class_ = 'link-comp link-comp--appearance-dark')
            lst_r = []
            for i in req:
                lst_r.append(i.text)
        except:
            req = ''
        try:
            salary = lst_r[-1]
        except:
            salary = ''
        try:
            spec = lst_r[0]
        except:
            spec = ''
        try:
            level = lst_r[1]
        except:
            level = ''
        try:
            skills = lst_r[2:-1]
        except:
            skills = ''          
           
        try:
            date_publish = data.find('time').attrs['datetime']
        except:
            date_publish = ''
            
        try:
            lc = data.find_all(class_ = 'inline-list')
            lst_l = []
            for i in lc[2]:
                lst_l.append(i.text)
            location = lst_l[0]
            working_day = lst_l[1]
            remote = lst_l[2]
        except:
            location = ''
            working_day = ''
            remote = ''
        try:
            employer_desc = data.find(class_ = 'vacancy-company__sub-title').text
        except:
            employer_desc = ''
        try:
            lc = data.find(class_ = 'vacancy-description__text').find_all(class_= 'style-ugc')
            lst = []
            for i in lc:
                lst.append(i.text)
        except:
            lc = ''
        try:
            respons = lst[0]
        except:
            respons = ''
        try:
            compens = lst[1]
        except:
            compens = ''
        
        try:
            add_info = lst[2]
        except:
            add_info = ''
   
            
        yield title, spec, salary, employer, level, skills, date_publish, location, working_day, remote,\
              employer_desc, respons, compens, add_info

habr = pd.DataFrame(columns = ['title', 'spec', 'employer', 'salary','level','skills', 'date_publish',\
                                'location', 'working_day', 'remote', 'employer_desc','respons','compens',
                              'add_info'])
for item in get_resume():
    resume_data = []
    for i in range(14):
        resume_data.append(item[i])
    habr.loc[len(habr.index)] = resume_data 